<a href="https://colab.research.google.com/github/arumshin-dev/python_conda_jupyter/blob/main/codeit/3_5_12_RAG_HuggingFace_Local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 미션 14: RAG 시스템 구현

## Part 3: HuggingFace 로컬 LLM으로 RAG 구축

**PyMuPDFLoader**로 문서를 로드하고, **HuggingFace 모델을 로컬 GPU에서 실행**하여 RAG 시스템을 구축합니다.

### RAG 파이프라인 개요

```
┌─────────────┐    ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
│  PDF 문서   │ -> │   Chunking  │ -> │  Embedding  │ -> │ Vector Store│
│ PyMuPDFLoader│    │ TextSplitter │    │ HuggingFace │    │    FAISS    │
│             │    │             │    │   (로컬)    │    │             │
└─────────────┘    └─────────────┘    └─────────────┘    └─────────────┘
                                                                │
┌─────────────┐    ┌─────────────┐    ┌─────────────┐          │
│    답변     │ <- │     LLM     │ <- │  Retriever  │ <────────┘
│             │    │   (로컬)    │    │   검색기     │
└─────────────┘    └─────────────┘    └─────────────┘
```

### Inference API vs 로컬 실행 비교

| 항목 | Inference API | 로컬 실행 |
|------|---------------|----------|
| **API 키** | 필요 | 불필요 |
| **비용** | 무료 tier 제한 | 완전 무료 |
| **속도** | 네트워크 의존 | GPU 성능 의존 |
| **GPU** | 불필요 | 필요 (VRAM ~14GB) |
| **오프라인** | 불가 | 가능 |

### ⚠️ 요구사항
- **GPU**: VRAM 14GB 이상 (Colab T4 무료 GPU 가능)
- **Colab**: 런타임 > 런타임 유형 변경 > T4 GPU 선택

---

## 1️⃣ 환경 설정

In [ ]:
# GPU 확인
!nvidia-smi

Wed Dec 10 04:46:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 라이브러리 설치 (로컬 실행용 추가 패키지)
!pip install -q pymupdf langchain langchain-huggingface langchain-community faiss-cpu
!pip install -q sentence-transformers transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 12.7 MB/s eta 0:00:00


In [ ]:
# 버전 확인
import torch
import langchain

print(f"LangChain 버전: {langchain.__version__}")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

LangChain 버전: 1.1.2
PyTorch 버전: 2.9.0+cu126
CUDA 사용 가능: True
GPU: Tesla T4
VRAM: 14.7 GB


In [ ]:
# 로컬 실행: API 토큰 불필요!
# (Inference API와 달리 토큰 입력 셀 삭제됨)
print("로컬 실행 모드 - API 토큰 불필요!")

로컬 실행 모드 - API 토큰 불필요!


In [ ]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline  # transformers 추가
import torch
import time

In [ ]:
# # PDF 파일 경로 설정
# PDF_PATH = "2024년_원천징수의무자를_위한_연말정산_신고안내.pdf"

# Google Colab에서 실행시 파일 업로드
from google.colab import files
uploaded = files.upload()

PDF_PATH = list(uploaded.keys())[0]

Saving 2024년_원천징수의무자를_위한_연말정산_신고안내.pdf to 2024년_원천징수의무자를_위한_연말정산_신고안내.pdf


---

## 2️⃣ 문서 로드 (PyMuPDFLoader)


In [ ]:
# PyMuPDFLoader로 문서 로드
print("="*60)
print("📄 PyMuPDFLoader로 문서 로드 중...")
print("="*60)

start_time = time.time()
loader = PyMuPDFLoader(PDF_PATH)
documents = loader.load()
load_time = time.time() - start_time

print(f"\n로드 완료!")
print(f"총 페이지 수: {len(documents)}")
print(f"소요 시간: {load_time:.2f}초")

📄 PyMuPDFLoader로 문서 로드 중...

로드 완료!
총 페이지 수: 426
소요 시간: 1.22초


In [ ]:
# 로드된 문서 샘플 확인
print("첫 번째 페이지 메타데이터:")
print(documents[0].metadata)

print("\n첫 번째 페이지 내용 (처음 500자):")
print(documents[0].page_content[:500])

첫 번째 페이지 메타데이터:
{'producer': 'ezPDF Builder Supreme', 'creator': '', 'creationdate': '2024-12-22T23:44:00+09:00', 'source': '2024년_원천징수의무자를_위한_연말정산_신고안내.pdf', 'file_path': '2024년_원천징수의무자를_위한_연말정산_신고안내.pdf', 'total_pages': 426, 'format': 'PDF 1.6', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-01-09T17:28:20+09:00', 'trapped': '', 'modDate': "D:20250109172820+09'00'", 'creationDate': "D:20241222234400+09'00'", 'page': 0}

첫 번째 페이지 내용 (처음 500자):
연말정산
신고안내
일 하나는 제대로 하는,
국민께 인정받는 국세청
2024. 12.
2
0
2
4
 
원
천
징
수
의
무
자
를
 
위
한
맞춤형 안내
간소화 서비스
일괄제공 서비스
발
간
등
록
번
호
11-1210000-000072-10


---

## 3️⃣ 텍스트 청킹 (Chunking)

(Inference API 버전과 동일)

In [ ]:
# 텍스트 분할기 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

print("TextSplitter 설정")
print(f"   - chunk_size: 1000")
print(f"   - chunk_overlap: 200")

TextSplitter 설정
   - chunk_size: 1000
   - chunk_overlap: 200


In [ ]:
# 문서 분할 실행
print("\n" + "="*60)
print("문서 청킹 중...")
print("="*60)

start_time = time.time()
chunks = text_splitter.split_documents(documents)
chunk_time = time.time() - start_time

print(f"\n청킹 완료!")
print(f"원본 문서: {len(documents)}개 → 청크: {len(chunks)}개")
print(f"평균 청크 수/페이지: {len(chunks)/len(documents):.1f}개")
print(f"소요 시간: {chunk_time:.2f}초")


문서 청킹 중...

청킹 완료!
원본 문서: 426개 → 청크: 634개
평균 청크 수/페이지: 1.5개
소요 시간: 0.03초


---

## 4️⃣ HuggingFace Embedding (로컬 실행)

### 💡 Embedding은 이미 로컬 실행!

`HuggingFaceEmbeddings`는 기본적으로 **로컬에서 실행**됩니다.  

In [ ]:
# HuggingFace Embedding 모델 초기화 (로컬 실행 - GPU 사용)
device = "cuda" if torch.cuda.is_available() else "cpu"

embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask",  # 동일한 한국어 특화 모델
    model_kwargs={"device": device},           # 🔄 GPU 사용
    encode_kwargs={"normalize_embeddings": True}
)

print("HuggingFace Embedding 모델 초기화 완료 (로컬)")
print(f"   - 모델: jhgan/ko-sroberta-multitask")
print(f"   - 디바이스: {device}")
print(f"   - 차원: 768")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HuggingFace Embedding 모델 초기화 완료 (로컬)
   - 모델: jhgan/ko-sroberta-multitask
   - 디바이스: cuda
   - 차원: 768


In [ ]:
# Embedding 테스트
test_text = "연말정산 의료비 공제 한도는 얼마인가요?"
test_embedding = embeddings.embed_query(test_text)

print("Embedding 테스트")
print(f"   - 입력 텍스트: '{test_text}'")
print(f"   - 벡터 차원: {len(test_embedding)}")
print(f"   - 벡터 샘플 (처음 5개): {test_embedding[:5]}")

Embedding 테스트
   - 입력 텍스트: '연말정산 의료비 공제 한도는 얼마인가요?'
   - 벡터 차원: 768
   - 벡터 샘플 (처음 5개): [-0.0190353374928236, -0.029213303700089455, 0.014034700579941273, -1.3819972082274035e-05, -0.017807548865675926]


---

## 5️⃣ Vector Store (FAISS)

(Inference API 버전과 동일)

In [ ]:
# FAISS Vector Store 생성
print("="*60)
print("FAISS Vector Store 생성 중...")
print("="*60)
print(f"\n임베딩할 청크 수: {len(chunks)}개")

start_time = time.time()
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)
embed_time = time.time() - start_time

print(f"\nVector Store 생성 완료!")
print(f"소요 시간: {embed_time:.2f}초")
print(f"처리 속도: {len(chunks)/embed_time:.1f} 청크/초")

FAISS Vector Store 생성 중...

임베딩할 청크 수: 634개

Vector Store 생성 완료!
소요 시간: 4.15초
처리 속도: 152.6 청크/초


---

## 6️⃣ 유사도 검색 테스트


In [ ]:
# 유사도 검색 테스트
query = "의료비 세액공제 한도"

print(f"검색 쿼리: '{query}'")
print("="*60)

similar_docs = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(similar_docs, 1):
    print(f"\n결과 {i} (페이지: {doc.metadata.get('page', 'N/A')})")
    print("-"*50)
    print(doc.page_content[:400] + "...")

검색 쿼리: '의료비 세액공제 한도'

결과 1 (페이지: 185)
--------------------------------------------------
원천징수의무자를 위한 
2024년 연말정산 신고안내
170
다. 의료비 세액공제(소법 §59의4 ②)
  근로소득이 있는 거주자가 기본공제대상자(나이 및 소득의 제한을 받지 않음)를 위하여 해당 
과세기간에 의료비를 지급한 경우 다음 금액의 15%(미숙아·선천성 이상아를 위해 지출한 
의료비는 20%, 난임시술비는 30%)에 해당하는 금액을 해당 과세기간의 종합소득산출세액에서
공제한다.
①근로자 본인, 과세기간 종료일 현재 65세 이상인 사람과 장애인을 위하여 지급한 의료비, 
미숙아·선천성 이상아를 위해 지출한 의료비, 난임시술비, 건강보험 산정특례 대상자로 
등록(재등록)된 자에 대한 의료비
   - 다만, ②의 의료비가 ‘총급여액 × 3%’에 미달하는 경우에는 그 미달금액을 차감
참고
○ 건강보험 산정...

결과 2 (페이지: 186)
--------------------------------------------------
Ⅵ. 세액감면(공제) 및 농어촌특별세
171
1) 공제 신청 시 제출서류
  근로자가 의료비 세액공제를 받기 위해서는 의료비영수증과 의료비지급명세서를 소득·세액
공제신고서에 첨부하여 제출하여야 한다.
① 의료비영수증
   ○｢의료법｣에 따른 의료기관 및 ｢약사법｣에 따른 약국에 지급한 의료비
      - ｢국민건강보험요양급여의 기준에 관한 규칙｣ 제7조 제1항에 따른 계산서·영수증, 동조 
제2항에 따른 진료비(약제비) 납입확인서 또는 ｢국민건강보험법｣에 따른 국민건강보험
공단의 이사장이 발행하는 의료비 부담 명세서
   ○  ｢노인장기요양보험법｣ 제40조 제1항 및 같은 조 제2항 제3호에 따른 장기요양급여비용
으로서 실제 지출한 본인일부부담금은 ‘장기요양급여비용 명세서’ 또는 ‘장기요양급여비
납부확인서...

결과 3 (페이지: 20)
-------------------

---

## 7️⃣ HuggingFace LLM 설정 (로컬 실행) 🔄

### 🔄 핵심 변경: HuggingFaceEndpoint → HuggingFacePipeline

| Inference API | 로컬 실행 |
|---------------|----------|
| `HuggingFaceEndpoint` | `HuggingFacePipeline` |
| API 서버 호출 | 로컬 GPU 실행 |
| 토큰 필요 | 토큰 불필요 |

### 💡 4-bit 양자화로 VRAM 절약
- 원본: ~14GB VRAM 필요
- 4-bit: ~6GB VRAM으로 실행 가능

In [ ]:
# HuggingFace LLM 로컬 실행 (동일한 모델: zephyr-7b-beta)
from transformers import BitsAndBytesConfig

MODEL_ID = "HuggingFaceH4/zephyr-7b-beta"  # 동일한 모델

print("="*60)
print(f"LLM 로컬 로드 중: {MODEL_ID}")
print("="*60)
print("모델 다운로드 및 로드 중... (최초 실행시 몇 분 소요)")

# 4-bit 양자화 설정 (VRAM 절약)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

# 모델 로드 (4-bit 양자화)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("\n모델 로드 완료!")

LLM 로컬 로드 중: HuggingFaceH4/zephyr-7b-beta
모델 다운로드 및 로드 중... (최초 실행시 몇 분 소요)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]


모델 로드 완료!


In [ ]:
# HuggingFacePipeline 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1000,
    temperature=0.1,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False
)

llm = HuggingFacePipeline(pipeline=pipe)

print("HuggingFacePipeline 생성 완료 (로컬 실행)")
print(f"   - 모델: {MODEL_ID}")
print(f"   - 양자화: 4-bit")
print(f"   - max_new_tokens: 1000")
print(f"   - temperature: 0.1")

Device set to use cuda:0


HuggingFacePipeline 생성 완료 (로컬 실행)
   - 모델: HuggingFaceH4/zephyr-7b-beta
   - 양자화: 4-bit
   - max_new_tokens: 1000
   - temperature: 0.1


In [ ]:
# LLM 단독 테스트
response = llm.invoke("안녕하세요, 간단히 자기소개 해주세요.")
print("LLM 테스트 응답:")
print(response)

LLM 테스트 응답:


Hello, please introduce yourself briefly.

저는 한국에서 출신하였으며, 2016년 그리니치 대학에서 컴퓨터 공학과를 마쳤습니다. 현재 애플에서 작업하고 있습니다. 주로 iOS 애플리케이션을 개발하고 있습니다.

I'm from South Korea and graduated from New York University in Computer Science in 2016. Currently working at Apple and mainly developing iOS applications.

저는 애플에서 일하면서도 여러분들의 아이디어와 피드백을 받아 새로운 애플리케이션을 만들어보려고 합니다. 또한 애플에서 제공하는 수강료를 받아 스위프T 언어에 대해 더욱 배우려고 합니다.

While working at Apple, I'd like to create new applications based on your ideas and feedback. Additionally, I'm learning more about Swift through the courses provided by Apple.


---

## 8️⃣ RAG 체인 구성 (LCEL 방식)


In [ ]:
# Retriever 생성
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

print("Retriever 생성 완료")
print(f"   - 검색 타입: similarity")
print(f"   - 검색 개수: 4")

Retriever 생성 완료
   - 검색 타입: similarity
   - 검색 개수: 4


In [ ]:
# 프롬프트 템플릿 정의 (Zephyr 모델 형식에 맞게)
template = """<|system|>
당신은 한국 세법 전문가입니다. 주어진 문서 내용을 바탕으로 질문에 정확하게 답변해주세요.
문서에 답변이 없는 경우, "제공된 문서에서 해당 정보를 찾을 수 없습니다"라고 답변하세요.
추측하지 말고, 문서에 있는 내용만 답변하세요. 한국어로 답변하세요.</s>
<|user|>
참고 문서:
{context}

질문: {question}</s>
<|assistant|>
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
# 문서 포맷팅 함수
def format_docs(docs):
    return "\n\n".join(
        f"[페이지 {doc.metadata.get('page', 'N/A')}]\n{doc.page_content}"
        for doc in docs
    )

In [ ]:
# LCEL 방식으로 RAG 체인 구성
rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ RAG 체인 구성 완료! (LCEL 방식)")
print("\n📋 체인 구성:")
print("   Query → Retriever → format_docs → Prompt → LLM (로컬) → Output")

✅ RAG 체인 구성 완료! (LCEL 방식)

📋 체인 구성:
   Query → Retriever → format_docs → Prompt → LLM (로컬) → Output


---

## 9️⃣ RAG 시스템 테스트

In [ ]:
def ask_question(question):
    """RAG 시스템에 질문하고 결과를 출력합니다."""
    print("="*70)
    print(f"질문: {question}")
    print("="*70)

    start_time = time.time()
    answer = rag_chain.invoke(question)
    response_time = time.time() - start_time

    print(f"\n답변:")
    print(answer)

    source_docs = retriever.invoke(question)
    print(f"\n참조 문서 ({len(source_docs)}개):")
    for i, doc in enumerate(source_docs, 1):
        page = doc.metadata.get('page', 'N/A')
        print(f"   {i}. 페이지 {page}: {doc.page_content[:80]}...")

    print(f"\n응답 시간: {response_time:.2f}초")
    print("\n")

    return answer

In [ ]:
# 테스트 질문 1: 의료비 공제
result1 = ask_question("의료비 세액공제 한도는 얼마인가요?")

질문: 의료비 세액공제 한도는 얼마인가요?

답변:
Based on the information provided in the text material, here is an answer to your question:

The limit for the deductible amount of medical expenses under the Medical Expense Deduction System is 700 million won per year (as of 2024). This limit applies to both the insured and their dependents, as well as individuals with disabilities or chronic diseases who are not covered by this system. The actual amount that can be deducted may vary depending on the individual's income level and other factors. For more detailed information, it is recommended to refer to the National Health Insurance Corporation's website or consult a tax professional. However, as a general guideline, the deductible amount is calculated based on the following formula:

- For those whose total annual income exceeds 3 million won: 3% of the total annual income
- For those whose total annual income does not exceed 3 million won: 3% of the total annual income up to 3 million won, plus an addi

In [ ]:
# 테스트 질문 2: 연장근로수당
result2 = ask_question("연장근로수당은 어떻게 계산하나요?")

질문: 연장근로수당은 어떻게 계산하나요?

답변:
답변: 연장근로수당은 연장근로(정상근무시간 50% 가산하여 지급)에 해당하는 수당과 연장근무시간(정상근무시간 4시간)에 해당하는 수당을 합산하여 계산합니다. 예를 들어, 정상근무시간은 8시간이고, 연장근로시간은 4시간이며, 연장근로수당은 5,000원이고, 연장근무시간 수당은 2,500원이라 합산하면, 연장근로수당은 7,500원이됩니다. (참고 문서: [페이지 83])

참조 문서 (4개):
   1. 페이지 83: 원천징수의무자를 위한 
2024년 연말정산 신고안내
68
   
참고
통상임금에 가산하여 지급받는 급여의 범위
근로기준법 제56조【연장·야간 및...
   2. 페이지 417: 총액에서 ｢근로기준법｣에 따른 연장근로·야간근로 또는 휴일근로를 
하여 통상임금에 더하여 받는 급여 및 ｢선원법｣에 따라 받는 생산수당
(비율급...
   3. 페이지 91: 원천징수의무자를 위한 
2024년 연말정산 신고안내
76
예규
○ 근로계약에 따른 근로의 대가를 금전 외의 것을 받기로 하였으나, 지급받기로 약...
   4. 페이지 63: 48
Ⅰ
근로소득
01 근로소득의 범위(소법 §20)
  근로소득은 고용관계 또는 이와 유사한 계약에 의하여 근로자가 근로를 제공하고 받는 모든...

응답 시간: 66.03초




In [ ]:
# 테스트 질문 3: 기본공제
result3 = ask_question("기본공제 대상자의 소득요건은 무엇인가요?")

질문: 기본공제 대상자의 소득요건은 무엇인가요?

답변:
기본공제 대상자는 70세 이상인 사람, 장애인, 배우자가 없는 사람으로서 기본공제 대상자인 직계비속·입양자가 있는 경우입니다. 이 경우, 해당 과세기간의 근로소득금액에서 공제됩니다. 추가공제는 여성이 있는 경우에 한해, 70세 이상인 사람, 장애인, 배우자가 없는 사람으로서 기본공제 대상자인 직계비속·입양자가 있는 경우입니다. 이 경우, 추가공제를 받을 수 있습니다. 배우자는 기본공제 대상자인 사람으로서 추가공제를 받을 수 없습니다.

참조 문서 (4개):
   1. 페이지 108: 93
Ⅲ
근로소득공제, 인적공제, 연금보험료공제
※ 인적공제와 연금보험료공제는 종합소득자(근로소득자 포함)가 받을 수 있는 소득공제
○ 총급여액...
   2. 페이지 119: 104
Ⅳ
특별소득공제(소법 §52)
✲특별소득공제는 ‘근로소득이 있는 거주자’가 받을 수 있으며, ｢조세특례제한법｣에 따른 ‘그 밖의 소득공제...
   3. 페이지 418: 부록5. 연말정산 주요 용어 설명
403
구분
용어
설  명
관련법령
27
인적공제
｢소득세법｣상 인적공제는 종합소득이 있는 거주자(자연인에 한...
   4. 페이지 5: ( = 근로소득금액 - 인적공제 - 연금보험료공제 - 특별소득공제 - 그 밖의 소득공제 + 소득공제 
종합한도초과액)
* ‌소득공제 종합한도초...

응답 시간: 60.14초




In [ ]:
# 테스트 질문 4: 식사대 비과세
result4 = ask_question("식사대 비과세 요건을 알려주세요.")

질문: 식사대 비과세 요건을 알려주세요.

답변:
답변: 식사대는 실비변상적인 급여에 해당하지 않으며, 월정액급여 계산 시 차감하지 않도록 주의해야 합니다. 하지만, 다른 근로자와 함께 일률적으로 급식수당 (20만원 이내 비과세)을 지급받고 있는 근로자가 야간근무 등 시간외근무를 하는 경우에 별도로 제공받은 식사·기타 음식물은 비과세되는 급여에 포함됩니다. 따라서, 식사대의 크기가 매월 변동되더라도 매월 계산되는 급여항목인 경우에는 급여총액에 포함됩니다. 예를 들어, 생산직 등에 종사하는 근로자의 야간수당 등은 소득세법§12 3 러에 해당하며, 비과세됩니다. 하지만, 식사대의 크기가 20만원 이하인 경우에는 소득세법§12 3 머에 해당하며, 비과세됩니다. 따라서, 식사대의 크기가 20만원 이하인 경우에는 주어진 문서에서 해당 정보를 찾을 수 없습니다. 그러나, 식사대의 크기가 20만원 초과하더라도 월정액급여 계산 시 차감하지 않도록 주의해야 합니다. 예를 들어, 생산직 등에 종사하는 근로자의 식사대의 크기가 23만원인 경우에는 비과세되는 급여항목인 경우에는 주어진 문서에서 해당 정보를 찾을 수 없습니다. 그러나, 식사대의 크기가 20만원 이하인 경우에는 주어진 문서에서 해당 정보를 찾을 수 있습니다. 따라서, 식사대의 크기가 20만원 초과하더라도 주어진 문서에서 해당 정보를 찾을 수 있는 경우에는 주어진 문서에서 해당 정보를 찾을 수 있습니다. 예를 들어, 생산직 등에 종사하는 근로자의 식사대의 크기가 22만원인 경우에는 주어진 문서에서 해당 정보를 찾을 수 있습니다. 그러나, 식사대의 크기가 20만원 초과하더라도 주어진 문서에서 해당 정보를 찾을 수 있는 경우에는 주어진 문서에서 해당 정보를 찾을 수 있습니다. 따라서, 식사대의 크기가 20만원 초과하더라도 주어진 문서에서 해당 정보를 찾을 수 있는 경우에는 주어진 문서에서 해당 정보를 찾을 수 있습니다. 예를 들어, 생산직 등에 종사하는 근로자의 식사대의 크기가 223만원인 경우에는 주어진 문서에서 해당 정

---

## 🔟 스트리밍 출력

In [ ]:
question = "연말정산 시 교육비 공제 대상은 무엇인가요?"

print(f"질문: {question}")
print("="*60)
print("\n답변 (스트리밍):")

for chunk in rag_chain.stream(question):
    print(chunk, end="", flush=True)

print("\n")

질문: 연말정산 시 교육비 공제 대상은 무엇인가요?

답변 (스트리밍):
연말정산 시에 교육비 공제 대상은 다음과 같습니다:

1. 교육비 (보험료, 의료비, 기부금): 세액공제 대상금액은 100만원입니다. 공제율은 다음과 같습니다:
   - 보험료: 12%
   - 장애인전용: 보장성보험 피보헌자 또는 수익자로 하는 장애인전용보장성보험료에 대해 15%
   - 의료비:
       - 본인: 전액
       - 장애인: 전액
       - 65세 이상자, 6세 이하자, 미숙아 및선천성이상아, 건강보험 산정 특례자, 난임시술비:
           - 3% 초과분: 총급여에 3% 이상인 경우에 대해 15%
           - 미숙아·선천성이상아 의료비: 20%
           - 난임시술비:
               - 미숙아·선천성이상아: 20%
               - 30%
2. 교육비:
   - 본인: 전액
   - 대학원, 대학, 시간제과정, 직업능력개발훈: 15%
   - 취학전 아동: 연 300만원
   - 초·중·고생: 연 900만원

3. 정치자금기부금: 정당기부 등에 대해 10만원 이하에 대해 100/110, 10만원 초과에 대해 15% 공제됩니다.
4. 고향사랑기부금: 지방자치단체에 대해 500만원까지 10만원 이하에 대해 100/110, 10만원 초과에 대해 15% 공제됩니다.
5. 특례기부금:
   - 국방헌금, 위문금품 등: 1천만원 이하에 대해 15%, 1천만원 초과에 대해 30% 공제됩니다.
   - 우리사주조합기부금: 우리사주조합원이 아닌 사람이 우리사주조합에 지출하는 기부금에 대해 30% 공제됩니다.
6. 일반기부금:
   - 종교단체에 기부한 금액이 있는 경우: 10년에 대해 10% + 종교단체 외에 지급한 금액 * 중 적은 금액에 대해 중 적은 금액에 대해 중 적은 금액에 대해 중 적은 금액에 대해 중 적은 금액에 대해 중 적은 금액에 대해 중 적은 금액에 대해 중 적은 금액에 대해 중 적은 금액에 대해 중

---

## 💡 로컬 vs Inference API 실행 요약

### 🔄 변경된 부분

```python
# 1. pip install 추가
!pip install transformers accelerate bitsandbytes

# 2. API 토큰 삭제
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = ...  # 불필요!

# 3. LLM 변경
# Inference API 버전
from langchain_huggingface import HuggingFaceEndpoint
llm = HuggingFaceEndpoint(repo_id="HuggingFaceH4/zephyr-7b-beta", ...)

# 로컬 실행 버전
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, quantization_config=bnb_config, ...)
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, ...)
llm = HuggingFacePipeline(pipeline=pipe)
```

### ✅ 변경되지 않은 부분
- 문서 로드 (PyMuPDFLoader)
- 청킹 (RecursiveCharacterTextSplitter)
- Embedding (HuggingFaceEmbeddings) - 원래 로컬 실행
- Vector Store (FAISS)
- **RAG 체인 구성 (LCEL)** - 완전히 동일!
- 프롬프트 템플릿

---

## 로컬 실행의 장점

1. **API 토큰 불필요**: 인증 없이 바로 사용
2. **오프라인 가능**: 인터넷 없이도 실행 (모델 다운로드 후)
3. **속도**: 네트워크 지연 없음
4. **프라이버시**: 데이터가 외부 서버로 전송되지 않음